# Assignment Netwerkanalyse

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: alleen de hoofdletter (dus voor de groep van Marx zou je `A` kiezen)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* De meeste opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Week 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import networkx as nx
import requests
from nose.tools import assert_equal, assert_count_equal
from itertools import combinations
from collections import defaultdict

# Deel -1

* Verzamelingen
    * Maak alle opgaven op [deze site](https://set.kuleuven.be/voorkennis/zomercursus/zomercursusZ/logica/verzamelingen_light). Dus ook die in sectie 2.1 en 2.1.1! 

# Deel 0: First Course Chap 1

*  First Course Chap 1, alle overige opgaven (dus vanaf 22) (plus opnieuw die je vorige week fout had)

1.22 
min degree = 2 <br>
max degree = 5 <br>
mean degree = 1,33 <br>
density = 0,6 <br>

1.23
No, they don't have to have the same minimum and maximum degree. The way that links are distributed could be different in both networks. 

They must have the same mean degree though, since the same amount of links is distributed over the same amount of nodes. The sum of links / amount of nodes will be the same.

1.24
With the increase of the average number of friends and the amount of users staying the same, the density will increase. 
This because the amount of links increased, but the maximum number of links stayed the same, which indicates that the density must be higher.

If both the number of users and the average number of friends increased, the density would stay the same.

1.25 <br>
33 million * 1000 = 33.000.000.000 links. <br>
max. number of links = 33 million * 100000 <br>

density = 33 miljard / (33 million * 100000) = 0.01 <br>
The network is sparse.

1.26 The density will stay the same, because the number of links, and max number of links will grow with the same factor.

# Deel 1: MacBeth

In [ ]:
def open_graph():
    """Loads the graphml"""
    if 'macbethcorrected.graphml' in os.listdir():
        return nx.read_graphml('macbethcorrected.graphml')
    elif os.path.exists('../../data/Week2'):
        return nx.read_graphml('../../data/Week2/macbethcorrected.graphml')
    elif os.path.exists('../../../data/Week2'):
        return nx.read_graphml('../../../data/Week2/macbethcorrected.graphml')

In [ ]:
MacB = open_graph()
nx.draw(MacB, with_labels=True)

## Vraag 1
 
In deze vraag gaan we kijken naar de vorm van deelgrafen in MacB. We vragen ons voor elke hoeveelheid knopen af, hoeveel verschillende "vormen" deelnetwerken met zoveel knopen er in MacB zitten.

Voorbeeld: met 2 knopen hebben we ook 2 "vormen" in MacB, namelijk de twee knopen onverbonden, en de twee knopen verbonden. Met drie knopen zijn er meer mogelijkheden (vogel ze zelf uit), en het is de vraag of we die ook allemaal terugzien in MacB. 

Alle deelnetwerken van dezelfde vorm noemen we isomorf. 

Geef voor `n in range(5)` het aantal niet isomorfe deelnetwerken in MacB met `n` knopen als een dict met het aantal knopen als sleutel en het aantal niet isomorfe deelnetwerken als waarde. 

**Hint:** teken voor elk aantal knopen eerst zelf alle mogelijke niet isomorfe netwerkjes die je met zoveel knopen kunt maken, en ga dan op zoek in de Macbeth graaf of je die allemaal kunt vinden.  Voor $n=2$ zijn er 2 mogelijke niet isomorfe netwerkjes: $\langle\{1,2\},\emptyset\rangle$ en $\langle\{1,2\},\{(1,2)\}\rangle$.

**Tip:** Je mag dit natuurlijk programmeren, maar het hoeft niet. 


In [ ]:
iso_types = {}

for n in range(5):
    combs = combinations(MacB.nodes(), n)
    unique_subgraphs = set()
    for c in combs:
        subgraph = MacB.subgraph(c)
        is_isomorphic = False
        for existing_subgraph in unique_subgraphs:
            if nx.is_isomorphic(subgraph, existing_subgraph):
                is_isomorphic = True
                break
        if not is_isomorphic:
            unique_subgraphs.add(subgraph)
    iso_types[n] = len(unique_subgraphs)

iso_types

In [ ]:
assert_equal(type(iso_types), dict)
assert_equal(len(iso_types),5)
assert_equal(set(iso_types.keys()),set(range(5)))

for n in iso_types:
    assert isinstance(iso_types[n], int)

## Vraag 2
Bereken voor ieder personage zijn "degree" en zijn "neighbour set". Stop dit in een dictionary genaamd `personage_dict` waar de key het personage is en de value een tuple (een paar in dit geval)  met eerst de degree en dan de neighbour set in de vorm van een lijst.

* Voorbeeld:
`{FirstWitch: (4, ['ALL', 'ThirdWitch', 'MACBETH', 'SecondWitch'])...}`
* Hint: gebruik weer een dict comprehensie. Dit kan in 1 regel.

In [ ]:
personage_dict = {x: (MacB.degree(x), list(MacB.neighbors(x))) for x in MacB}
personage_dict

In [ ]:
assert_equal(type(personage_dict), dict)
assert_equal(type(list(personage_dict.values())[0]), tuple)
assert_equal(type(list(personage_dict.values())[0][0]), int)
assert_equal(type(list(personage_dict.values())[0][1]), list)
assert_equal(type(list(personage_dict.values())[0][1][0]), str)


## Vraag 3
Maak de lijst met alle maximale cliques, die meer dan 3 knopen bevatten, in de Macbeth graaf.

Geef een maximale clique weer als een lijst van de knopen (dus een lijst in een lijst). Noem deze `cliques`

Zorg dat alle lijsten alfabetisch gesorteerd zijn, ook de buitenste lijst. Dus je krijgt:

```
[['ALL', 'BANQUO', 'MACBETH', 'MALCOLM'],
 ['ALL', 'FirstWitch', 'MACBETH', 'SecondWitch', 'ThirdWitch'],
 ['ALL', 'LENNOX', 'MACBETH', 'MALCOLM', 'ROSS'],
 ....]
 ```

* Hint: gebruik `sorted` om te sorteren en natuurlijk doe je dit met een list comprehensie. 1 regel is genoeg.
* HINT: bedenk wat een maximale kliek in een netwerk is. (het driehoek netwerk heeft dus maar 1 maximale kliek, zichzelf, ook al zitten er meerdere klieks in (alle paartjes zijn ook klieks bijvoorbeeld))

In [ ]:
cliques = sorted([sorted(i) for i in (nx.find_cliques(MacB)) if len(i)>=3])

cliques

In [ ]:
assert_equal(type(cliques), list)
assert_equal(type(cliques[0]), list)
# elk element in zo'n kliek is een knoop
for L in cliques:
    for n in L:
        assert_equal(n in MacB,True,msg= 'elk element in zon kliek is een knoop')     

## Vraag 4
 
  
Hoeveel  deelnetwerken  in het MacBeth netwerk hebben een dichtheid (_density_) van 1?

Hint: kijk eens wat er gebeurt als je `nx.density` gebruikt op subgraven van verschillende lengtes.

In [ ]:
deelnetwerken = 0
deelnetwerken = len([c for c in nx.enumerate_all_cliques(MacB) if len(c) > 1])
deelnetwerken

In [ ]:
assert isinstance(deelnetwerken,int), "We vragen om een geheel getal."

# Deel 2: Easley and Kleinberg

## Vraag 5
Maak opdracht [3.7.1 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf). 
Geef je antwoord in de cell hieronder

Triadic closure is een systeem als 2 mensen die vrienden zijn een andere vriend hebben dat die dan connecten. Dit komt in de praktijk bij de vriendensuggesties van facebook en instagram voor.

## Vraag 6
Maak opdracht [3.7.2 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf).

Geef het antwoord als `'S'` of `'W'` aan de variabele `a`

In [ ]:
a = 'S'

a

In [ ]:
assert_equal(a == 'W' or a == 'S', True)


## Vraag 7
Maak opdracht [3.7.3 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf).

Geef alle nodes die voldoen in hoofdletters, als lijst van strings aan variabele `b`.

Voorbeeld: `x = ['A', 'B'...]`

In [ ]:
b = ['A','B','D',]

b

In [ ]:
assert_equal(type(b), list)
for x in b:
    if x not in ['A', 'B', 'C', 'D', 'E']:
        raise AssertionError("Een van de knopen in je lijst bestaat niet")


## Vraag 8
Maak opdracht [3.7.4 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf).

Geef alle nodes die niet voldoen in hoofdletters, als lijst van strings aan variabele `c`.

Voorbeeld: `x = ['A', 'B']`

In [ ]:
c = ['C','E']


c

In [ ]:
assert_equal(type(c), list)
for x in c:
    if x not in ['A', 'B', 'C', 'D', 'E']:
        raise AssertionError("Een van de knopen in je lijst bestaat niet")


## Vraag 9
Maak opdracht [3.7.5 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf).

Geef alle nodes die voldoen in hoofdletters, als lijst van strings aan variabele `d`.

Voorbeeld: `x = ['A', 'B'...]`

In [ ]:
d = ['A','B', 'C']


d

In [ ]:
assert_equal(type(d), list)
for x in d:
    if x not in ['A', 'B', 'C', 'D', 'E']:
        raise AssertionError("Een van de knopen in je lijst bestaat niet")


# Deel 3: Strong Triadic Closure

In [ ]:
# Hier staat een netwerk waarmee je je code kunt testen
# Je kunt ook zelf dingen veranderen in deze cel

G = nx.Graph()
G.add_edges_from([('D', 'E'), ('C', 'F'), ('A', 'F'), ('G', 'K'), ('G', 'H'),
                  ('H', 'B'), ('A', 'B')],
                 tie='w')
G.add_edges_from([('D', 'C'), ('D', 'A'), ('C', 'A'), ('E', 'A'), ('E', 'C'),
                  ('F', 'J'), ('F', 'G'), ('J', 'G'), ('K', 'H')],
                 tie='s')

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
my_edge_labels = {(
    u,
    v,
): d['tie']
                  for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=my_edge_labels);

In [ ]:
# Hier staat een netwerk waarmee je je code kunt testen
# Je kunt ook zelf dingen veranderen in deze cel

G = nx.Graph()
G.add_edges_from([('D', 'E'), ('C', 'F'), ('A', 'F'), ('G', 'K'), ('G', 'H'),
                  ('H', 'B'), ('A', 'B')],
                 tie='w')
G.add_edges_from([('D', 'C'), ('C', 'A'), ('E', 'A'), ('E', 'C'),
                  ('F', 'J'), ('F', 'G'), ('J', 'G'), ('K', 'H')],
                 tie='s')

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
my_edge_labels = {(
    u,
    v,
): d['tie']
                  for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=my_edge_labels);

## Vraag 10

Schrijf de Python functie `STC_node(G, n)` die voor een knoop `n` in een willekeurige invoer  graaf `G`, met sterke en zwakke relaties zoals hierboven, uitzoekt of die knoop voldoet aan Strong Triadic Closure.

* Als node `n` voldoet aan de Strong Triadic Closure eigenschap return dan de lege lijst `[]` .
* Zo niet geef dan de verzameling van alle driehoeken waarvan `n` 1 van de knopen is en die niet voldoen aan de STC eigenschap. 
    * De verzameling heeft de vorm `{frozenset({'A', 'C', 'D'}), frozenset({'A', 'E', 'D'})}`. 
        * **waarbij `{'A', 'C', 'D'}` een `frozenset` is**
        * Die maak je heel makkelijk met `frozenset(('A', 'C', 'D'))`
        * Je moet frozensets gebruiken omdat verzamelingen geen immutables mogen bevatten
    *  Dus de "ongesloten driehoekjes" representeren we als verzamelingen

1.            Check dat [Figuur 3.5](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf) inderdaad voldoet aan de STC.    Dit is het netwerk `G` wat boven gedefinieerd is.       
2.            Verander wat in Fig 3.5 zodat STC niet meer opgaat en draai je programma opnieuw. Doe dit door een copy van het netwerk `G` te maken. 

#### Hint: 
Zoals te zien in voorbeeldgraaf `G` hebben alle edges het attribuut `tie`. Deze kun je opvragen door `G[u][v]['tie']` voor edge `(u,v)`

In [ ]:
def STC_node(G, v):
    # Strong Triadic Closure Property at each node: if the node has strong ties
    # to two neighbors, then these neighbors must have at least a weak tie between them\
    
    #WRITE YOUR CODE HERE
# test
    strong_neighbors = {n for n in G.neighbors(v) if G[n][v]['tie'] == 's'}
    
    non_stc = {frozenset({v,n1,n2}) for n1 in strong_neighbors for n2 in strong_neighbors
              if (n1 != n2) and not G.has_edge(n1,n2)}
    
    return non_stc

for N in G:
    print(N, not bool(STC_node(G,N)), STC_node(G,N))

In [ ]:
 
assert_equal(type(STC_node(G, 'A')) == set , True)



## Vraag 11

Schrijf nu de Python functie `STC_graph(G)` die voor de hele graaf `G` uitzoekt of `G` voldoet aan Strong Triadic Closure (dit betekent dat elke knoop er aan voldoet).


* Als G voldoet aan de Strong Triadic Closure eigenschap return dan `True`.
* Zo niet return dan net zoals bij `STC_node` voor `STC_graph(G)`, de lijst met alle driehoeken die een tegenvoorbeeld voor STC zijn.   Ook hier mogen geen dubbelen in zitten.


          
#### Hint:
Je gebruikt natuurlijk `STC_node(G, x)` om `STC_graph(G)` te bepalen. Want de laatste is waar als de eerste waar is voor elke knoop in `G`.

In [ ]:
def STC_graph(G):
    #WRITE YOUR CODE HERE
    results = [STC_node(G, node) for node in G]

    return set(frozenset(triangle) for result in results for triangle in result)

STC_graph(G)


        
        
    

In [ ]:
if STC_graph(G) == True:
    pass
elif type(STC_graph(G)) == list and type(STC_graph(G)[0]) == tuple and len(
        STC_graph(G)[0]) == 3 and type(STC_graph(G)[0][0]) == str:
    pass
else:
    raise AssertionError("Lees de opdracht goed")
assert_equal(type(STC_graph(G)) == list or type(STC_graph(G)) == bool, True)
